In [3]:
# hit API 
import requests

# parsing data
import pandas as pd
from datetime import date

# scraping progress
from tqdm import tqdm

# connecting to db
import pymongo
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [26]:
# reading all BL number from google sheet
from acquiringbl import takingBL
bl_list = takingBL("DANMAR LINES")

In [14]:
# key to map key from json request to key used in db
Mapping = { "Estimated Vessel Departure"  : "ETD",
            "Actual Vessel Departure" : "ATD",
            "Estimated Vessel Arrival" : "ETA",
            "Actual Vessel Arrival" : "ATA"}

In [15]:
bl_list = ['BDOA03046']

In [29]:
list_of_dict = []
sign = False

for i, bls in enumerate(tqdm(bl_list)):
    try:
    
        print("BL in Process: " ,bls)
        
        # hitting API
        url = "https://api.dhl.com/dgff/transportation/shipment-tracking"
        querystring = {"housebill": bls}
        headers = {
            "content-type": "application/json",
            "DHL-API-Key": "qEHPTpIAc13hHfjB6H6gJQ0vEfmP6GhA"
        }

        # pasting json response
        response = requests.request("GET", url, headers=headers, params=querystring)
        json_data = response.json() if response and response.status_code == 200 else None

        temp_dict={}
        
        # list of containers in 1 bl
        list_of_containers=[]
        for i in json_data['ShipmentTracking']['Shipment']['TransportUnits']['TransportUnit']:
            list_of_containers.append(i['TransportUnitID'])

        print(list_of_containers)

        # parsing every data per container
        for c, container in enumerate(tqdm(list_of_containers)):
            current_dict={}
            current_dict.update({"Liners" : "Danmar Lines / DHL"})
            current_dict.update({"BL Number" : bls})
            current_dict.update({"Container Number" : container})
            current_dict.update({"From" : json_data['ShipmentTracking']['Shipment']['Origin']['LocationName'][:json_data['ShipmentTracking']['Shipment']['Origin']['LocationName'].index(",")]})
            current_dict.update({"To" : json_data['ShipmentTracking']['Shipment']['Destination']['LocationName']})
            
            for j, status_dict in enumerate (json_data['ShipmentTracking']['Shipment']['Timestamps']['Timestamp']):
                status_name = status_dict.get('TimestampDescription')
                status_date = status_dict.get('TimestampDateTime')
                case_data = {status_name:status_date[:10]}
                temp_dict.update(case_data)
                
            for old_key, new_key in Mapping.items():
                try:
                    current_dict[new_key] = temp_dict[old_key]
                except Exception as e:
                    current_dict[new_key] = None
                    print("THIS IS ERROR====>",e)
                    list_of_dict.append(current_dict)
                    sign = True

            if sign == False:
                list_of_dict.append(current_dict)
            print(bls, container,"done")
    except Exception as e:
        print(bls, "THIS IS ERROR====>",e)

  0%|          | 0/5 [00:00<?, ?it/s]

BL in Process:  BDOA02975
['KMTU9350991', 'BMOU6243329', 'FCIU7542981', 'TXGU5033010', 'SEGU5574968']


 20%|██        | 1/5 [00:00<00:01,  3.26it/s]

BDOA02975 KMTU9350991 done
BDOA02975 BMOU6243329 done
BDOA02975 FCIU7542981 done
BDOA02975 TXGU5033010 done
BDOA02975 SEGU5574968 done
BL in Process:  BDOA02980
['FFAU1035730', 'BMOU6360773', 'FFAU1092220', 'TXGU6053599', 'FFAU3978901']


 40%|████      | 2/5 [00:00<00:00,  3.44it/s]

...

BDOA02994 FFAU3036041 done
BL in Process:  BDOA03033
['CSNU7170185']


 80%|████████  | 4/5 [00:01<00:00,  4.22it/s]

BDOA03033 CSNU7170185 done
BL in Process:  BDOA03046
['FFAU3210739']


100%|██████████| 5/5 [00:01<00:00,  4.10it/s]

THIS IS ERROR====> 'Actual Vessel Arrival'
BDOA03046 FFAU3210739 done


In [30]:
list_of_dict

[{'Liners': 'Danmar Lines / DHL',
  'BL Number': 'BDOA02975',
  'Container Number': 'KMTU9350991',
  'From': 'Jakarta',
  'To': 'Cat Lai',
  'ETD': '2023-09-22',
  'ATD': '2023-09-24',
  'ETA': '2023-09-27',
  'ATA': '2023-09-30'},
 {'Liners': 'Danmar Lines / DHL',
  'BL Number': 'BDOA02975',
  'Container Number': 'BMOU6243329',
  'From': 'Jakarta',
  'To': 'Cat Lai',
  'ETD': '2023-09-22',
  'ATD': '2023-09-24',
  'ETA': '2023-09-27',
  'ATA': '2023-09-30'},
 {'Liners': 'Danmar Lines / DHL',
  'BL Number': 'BDOA02975',
  'Container Number': 'FCIU7542981',
  'From': 'Jakarta',
  'To': 'Cat Lai',
  'ETD': '2023-09-22',
  'ATD': '2023-09-24',
  'ETA': '2023-09-27',
  'ATA': '2023-09-30'},
 {'Liners': 'Danmar Lines / DHL',
  'BL Number': 'BDOA02975',
  'Container Number': 'TXGU5033010',
  'From': 'Jakarta',
  'To': 'Cat Lai',
  'ETD': '2023-09-22',
  'ATD': '2023-09-24',
  'ETA': '2023-09-27',
  'ATA': '2023-09-30'},
 {'Liners': 'Danmar Lines / DHL',
  'BL Number': 'BDOA02975',
  'Contain

In [7]:
# connect to mongodb
from mongoinit import mongo_table_initiation, insert_many_mongo

mongo_table_initiation()
insert_many_mongo(list_of_dict)

Today's Collection Name ===>  all_tracking_Oct-25-2023
Table initiation complete
Inserting Many Complete!!


In [ ]:
# exporting scraped data to excel
df = pd.DataFrame(list_of_dict)
excel_file_path = 'export excel/DHL.xlsx'
df.to_excel(excel_file_path, index=False)